In [ ]:
설계 개요
Character는 플레이어와 몬스터의 공통 부모 클래스입니다. 따라서 다음 원칙을 따릅니다.

이름, 전투 스탯(레벨, 체력, 공격력, 방어력)을 공통 속성으로 관리

“살아있는지”, “피해를 받는 로직”, “공격 로직”을 공통 메서드로 정의

랜덤 데미지는 게임성을 위해 random 모듈 사용

In [ ]:
Character 클래스 코드

In [ ]:
import random

class Character:
    def __init__(self, name, level, hp, attack, defense):
        self.name = name
        self.level = level
        self.hp = hp
        self.attack = attack
        self.defense = defense


In [ ]:
핵심 포인트

모든 캐릭터가 가져야 할 최소 단위의 정보만 포함

체력(hp)은 현재 체력으로 관리

In [ ]:
is_alive 메서드

In [ ]:
    def is_alive(self):
        return self.hp > 0

In [ ]:
설명

체력이 0보다 크면 생존

전투 종료 조건 및 자동사냥 흐름 제어에 사용됨

반환 타입은 bool

In [ ]:
take_damage 메서드

In [ ]:
    def take_damage(self, damage):
        actual_damage = damage - self.defense
        if actual_damage > 0:
            self.hp -= actual_damage

In [ ]:
특이사항

방어력이 데미지보다 크면 피해 없음

실제 피해량(actual_damage) 개념을 분리하여 계산

체력이 음수가 되는 것은 허용 (is_alive에서 판별)

In [ ]:
Q2. Player 클래스와 Monster 클래스

In [ ]:
class Player(Character):
    def __init__(self, name):
        super().__init__(name, level=1, hp=100, attack=25, defense=5)
        self.experience = 0


In [ ]:
특이사항

super()를 통해 Character 초기화

Player만의 고유 속성으로 experience 추가

In [ ]:
경험치 획득 메서드

In [ ]:
    def gain_experience(self, amount):
        self.experience += amount

In [ ]:
전투 보상 처리 시 사용

경험치 계산 책임을 Player 내부로 한정

In [ ]:
레벨업 메서드

In [ ]:
    def level_up(self):
        while self.experience >= 50:
            self.experience -= 50
            self.level += 1
            self.attack += 10
            self.defense += 5

In [ ]:
중요한 설계 포인트

while 사용 이유 → 한 번의 전투로 여러 레벨업 가능

체력은 유지 (자동사냥 난이도 유지 목적)

공격력과 방어력만 성장

In [ ]:
Monster 클래스 설계 개요
Monster는 레벨에 비례한 랜덤 스탯을 가집니다.

같은 레벨이라도 매번 다른 몬스터 생성

자동사냥의 반복성을 줄이기 위함

In [ ]:
Monster 클래스 코드

In [ ]:
class Monster(Character):
    def __init__(self, name, level):
        hp = random.randint(10, 30) * level
        attack = random.randint(5, 20) * level
        defense = random.randint(1, 5) * level
        super().__init__(name, level, hp, attack, defense)

In [ ]:
특이사항

스탯 계산을 super() 호출 이전에 수행

레벨이 높아질수록 난이도가 자연스럽게 상승

몬스터 이름과 레벨을 외부에서 제어 가능

In [ ]:
Q3. battle 함수

In [ ]:
설계 개요
battle 함수는 자동사냥의 핵심 루프입니다.

플레이어와 몬스터가 번갈아 공격

둘 중 하나라도 쓰러지면 즉시 종료

결과에 따라 보상 또는 패배 메시지 출력

In [ ]:
battle 함수 코드

In [ ]:
def battle(player, monster):
    while player.is_alive() and monster.is_alive():
        player.attack_target(monster)
        if monster.is_alive():
            monster.attack_target(player)

    if player.is_alive():
        exp_reward = monster.level * 20
        player.gain_experience(exp_reward)
        player.level_up()
        print("전투 승리!")
    else:
        print("전투 패배..")


In [ ]:
핵심 동작 설명 전투 루프
플레이어 선공

몬스터가 살아있을 경우에만 반격

불필요한 “죽은 상태에서 공격” 방지

승리 처리

경험치 = 몬스터 레벨 × 20

경험치 획득 후 즉시 레벨업 판정

자동사냥 특성상 사용자 개입 없음

패배 처리

메시지 출력만 수행

이후 흐름 제어는 외부에서 담당 가능

In [ ]:
전체 구조 요약

구성 요소	           역할
Character	      공통 전투 로직 추상화
Player	          성장 가능한 사용자 캐릭터
Monster	          레벨 기반 랜덤 적
battle	          자동 전투 및 보상 처리
